In [2]:
import sim
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Set up robot

In [4]:
sim.simxFinish(-1)  # Close opened connections
clientID = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim

if clientID != -1:
    print('Connected')

    # Now try to retrieve data in a blocking fashion (i.e. a service call):
    res, objs = sim.simxGetObjects(clientID, sim.sim_handle_all, sim.simx_opmode_blocking)

    print('Simulation time in milliseconds: ', sim.simxGetLastCmdTime(clientID))
    
    # Get Object position
    # I took a look at each robotic arm in CoppeliaSim, and the only one that does not have a
    # pre-determined path to follow is the "IRB140" robotic arm.  Generally every robotic arm
    # In coppeliaSim works in the same way, in that the robot has a sphere associated with it
    # that the end of the arm follows.  In this case, it's called "IRB140_manipulationSphere"
    robot_name = "IRB140_manipulationSphere"
    err_code, cuboid = sim.simxGetObjectHandle(clientID, robot_name, sim.simx_opmode_blocking)
    res, position = sim.simxGetObjectPosition(clientID, cuboid, -1, sim.simx_opmode_blocking)        

    # Now close the connection to CoppeliaSim:
    sim.simxGetPingTime(clientID)
    sim.simxFinish(clientID)
    print('Disconnected')
else:
    print('Failed connecting to remote API server')


Failed connecting to remote API server


In [8]:
class robot():
    
    def __init__(self, frame_name, motor_names=[], client_id=0):  
        # If there is an existing connection
        if client_id:
                self.client_id = client_id
        else:
            self.client_id = self.open_connection()
            
        self.motors = self._get_handlers(motor_names) 
        
        # Robot frame
        self.frame =  self._get_handler(frame_name)
            
        
    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        self.client_id = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim 
        
        if clientID != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return clientID
        
    def close_connection(self):    
        sim.simxGetPingTime(self.client_id)  # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxFinish(self.client_id)  # Now close the connection to CoppeliaSim:
        print('Connection closed')
    
    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0         
        
    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler
    
    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)
        
        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id, 
                                                      motor, vel, sim.simx_opmode_streaming)      
            
    def set_position(self, position, relative_object=-1):
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)        
        sim.simxSetObjectPosition(clientID, self.frame, relative_object, position, sim.simx_opmode_oneshot)
        
    def simtime(self):
        return sim.simxGetLastCmdTime(self.client_id)
    
    def get_position(self, relative_object=-1):
        # Get position relative to an object, -1 for global frame
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)
        res, position = sim.simxGetObjectPosition(self.client_id, self.frame, relative_object, sim.simx_opmode_blocking)        
        return array(position)
    
    def get_object_position(self, object_name):
        # Get Object position in the world frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, -1, sim.simx_opmode_blocking)
        return array(position)
    
    def get_object_relative_position(self, object_name):        
        # Get Object position in the robot frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, self.frame, sim.simx_opmode_blocking)
        return array(position)
    
# defining robot positions
# all of these positions are reversible, so this can be run regardless of whether the robot
# is on my right, my left, or straight forward.  Since I'm left handed, default is for the
# robot to be on the left, with reverse being if the robot is on the right.

# x_pos and y_pos are the coordinates of the base robot.

    def set_robot_lamp(self, x_pos, y_pos):
        # "lamp" is having the robotic arm standing straight up, can't be reversed
        x = x_pos
        y = y_pos
        z = 1.500
        return [x, y, z]

    def set_robot_light(self, x_pos, y_pos, reverse):
        # "light" is having the robotic arm facing downwards at a 45 degree angle, as if it's holding a flashlight
        # on something on a table.
        if reverse == False:
            x = x_pos + 0.650
            y = y_pos + 0.275
        else:
            x = x_pos + 0.650
            y = y_pos - 0.275
        z = 0.420
        return [x, y, z]

    def set_robot_hold(self, x_pos, y_pos, reverse):
        # "hold" is having the robotic arm facing straight forward and slightly up, as if it's holding a flashlight
        # on my face, or as if it's holding a book in front of me
        if reverse == False:
            x = x_pos + 0.165
            y = y_pos - 0.500
        else:
            x = x_pos + 0.165
            y = y_pos + 0.500
        z = 0.850
        return [x, y, z]
        
    def set_robot_pod(self, x_pos, y_pos, reverse):
        # "pod" is having the robotic arm facing towards me and slightly up, as if it's holding a book
        # off to the side that I'm reading
        if reverse == False:
            x = x_pos + 0.500
            y = y_pos - 0.260
        else:
            x = x_pos + 0.500
            y = y_pos + 0.260
        z = 0.850
        return [x, y, z]
        
    def set_robot_free(self, x_pos, y_pos, new_x, new_y, new_z):
        # "free" is a catch all.  It can set the robot to any position.  This is mostly used for
        # if I were to want the robot to move so I can acccess a different point on what it's holding
        
        x = x_pos + new_x
        y = y_pos + new_y
        z = new_z
        return [x, y, z]


In [11]:
# Joint name in coppelia
motor_names = []
r = robot('IRB140_manipulationSphere', motor_names)  # Create an instance of our robot

# Initial locations
robot_position = r.get_position()
print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

desired_position = r.set_robot_light(0, 0, False)

# Time interval
time_steps = linspace(0, 20, 100)

distance = (desired_position - robot_position) / len(time_steps)
for t in time_steps:
    # Sensing
    robot_position = r.get_position()
    
    robot_position = [robot_position[0] + distance, robot_position[1] + distance, robot_position[2] + distance]
    
    
    time.sleep(20/100)

r.close_connection()  # End

Connection failed
Robot position: (0.00, 0.00) 


TypeError: only size-1 arrays can be converted to Python scalars